# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
     - 8.382332347441762
- **Question 2: What is the 13th number yielded**
    - 3.605551275463989

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [30]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

final_val = 0

# generator

for sqrt_value in generator:
    print(generator)
    final_val += sqrt_value
    print(sqrt_value)
    print(f"final value: {final_val}")



<generator object square_root_generator at 0x7fc3c8f66200>
1.0
final value: 1.0
<generator object square_root_generator at 0x7fc3c8f66200>
1.4142135623730951
final value: 2.414213562373095
<generator object square_root_generator at 0x7fc3c8f66200>
1.7320508075688772
final value: 4.146264369941973
<generator object square_root_generator at 0x7fc3c8f66200>
2.0
final value: 6.146264369941973
<generator object square_root_generator at 0x7fc3c8f66200>
2.23606797749979
final value: 8.382332347441762


In [25]:
generator = square_root_generator(limit)
list(generator)

[1.0, 1.4142135623730951, 1.7320508075688772, 2.0, 2.23606797749979]

In [26]:
generator = square_root_generator(limit)
len(list(generator))

5

In [27]:
generator = square_root_generator(limit=13)
list(generator)[12]

3.605551275463989

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [42]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [76]:
# Importing the DLT library
import dlt

# Create a DLT pipeline for the first generator `people_1`
# The pipeline is set to load data into a DuckDB database with the dataset named 'people'
people_1_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# Run the pipeline for the first generator, creating or replacing the table 'people'
info = people_1_pipeline.run(people_1(),
                             table_name="people",
                             write_disposition="replace")

print(f"{info}\n\n")

# show outcome

import duckdb

conn = duckdb.connect(f"{people_1_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{people_1_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


people1 = conn.sql("SELECT * FROM people").df()
display(people1)


sums = conn.sql("SELECT sum(age) FROM people").df()
display(sums)

Pipeline dlt_ipykernel_launcher load step completed in 0.16 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/alainamartinez/Desktop/de-zoomcamp/de-zoomcamp/workshops/dlt_workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1712238743.441965 is LOADED and contains no failed jobs


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1712238743.441965,ekfI3IQyQkKzEA,None
1,2,Person_2,27,City_A,1712238743.441965,NuVbr49i0QwUnQ,None
2,3,Person_3,28,City_A,1712238743.441965,sZalj/HA898YuA,None
3,4,Person_4,29,City_A,1712238743.441965,/aeKkVxnA9CyYw,None
4,5,Person_5,30,City_A,1712238743.441965,jzDdD307CX4Ngg,None


,sum(age)
0,140.0


,sum(age)
0,140.0


sum of all ages = 140

#### append the 2nd generator to the first table.

In [36]:
# Create a DLT pipeline for the 2nd generator `people_2`
# The pipeline is set to load data into a DuckDB database with the dataset named 'people'
people_2_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# Run the pipeline for the first generator, creating or replacing the table 'people'
info = people_2_pipeline.run(people_2(),
                             table_name="people",
                             write_disposition="append")

print(f"{info}\n\n")

Pipeline dlt_ipykernel_launcher load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/alainamartinez/Desktop/de-zoomcamp/de-zoomcamp/workshops/dlt_workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1712238177.136405 is LOADED and contains no failed jobs




In [37]:
display(conn.sql("SELECT * FROM people").df())

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1712237947.599991,evTEuICqj0dPSg,None
1,2,Person_2,27,City_A,1712237947.599991,N8ZCBkOrdF507Q,None
2,3,Person_3,28,City_A,1712237947.599991,bUMA2VhUPZhNtA,None
3,4,Person_4,29,City_A,1712237947.599991,dHfcwg2NCo36Ow,None
4,5,Person_5,30,City_A,1712237947.599991,3DsC7D+sPt8bzw,None
5,3,Person_3,33,City_B,1712238177.136405,mZh1Djrhx8ApVw,Job_3
6,4,Person_4,34,City_B,1712238177.136405,/olR4yh+0Lt/fQ,Job_4
7,5,Person_5,35,City_B,1712238177.136405,7EqUpZ8Z8XnK9g,Job_5
8,6,Person_6,36,City_B,1712238177.136405,pDD4Te3lxzzHSA,Job_6
9,7,Person_7,37,City_B,1712238177.136405,rwJov5sFJ0cbCQ,Job_7


In [39]:
age_sum = display(conn.sql("SELECT sum(age) FROM people").df())
print(f"the sum of all ages with the 2 generators is {age_sum}")

,sum(age)
0,353.0


the sum of all ages with the 2 generators is None


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


### trying again...

In [80]:
# Importing the DLT library
import dlt

# Create a DLT pipeline for the first generator `people_1`
# The pipeline is set to load data into a DuckDB database with the dataset named 'people'
people_1_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_v2')

# Run the pipeline for the first generator, creating or replacing the table 'people'
info = people_1_pipeline.run(people_1(),
                             table_name="people_v2",
                             write_disposition="replace",
                             primary_key="id")

print(f"{info}\n\n")

# show outcome

import duckdb

conn = duckdb.connect(f"{people_1_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{people_1_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Pipeline dlt_ipykernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_v2
The duckdb destination used duckdb:////Users/alainamartinez/Desktop/de-zoomcamp/de-zoomcamp/workshops/dlt_workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1712307792.6355572 is LOADED and contains no failed jobs


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_v2           │
└─────────────────────┘

In [81]:
conn.sql("SELECT * FROM people_v2").df()

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1712307792.6355572,GmyOAhlvTl+vkw
1,2,Person_2,27,City_A,1712307792.6355572,WvU+btVlC36sow
2,3,Person_3,28,City_A,1712307792.6355572,84s2IaEenF1h7w
3,4,Person_4,29,City_A,1712307792.6355572,wjw2ZcfgbJzYcQ
4,5,Person_5,30,City_A,1712307792.6355572,pDTIww6XDlGXpA


now merge.

In [82]:
# Create a DLT pipeline for the 2nd generator `people_2`
# The pipeline is set to load data into a DuckDB database with the dataset named 'people'
people_2_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_v2')

# Run the pipeline for the first generator, creating or replacing the table 'people'
info = people_2_pipeline.run(people_2(),
                             table_name="people_v2",
                             write_disposition="merge",
                             primary_key="id")

print(f"{info}\n\n")

Pipeline dlt_ipykernel_launcher load step completed in 0.38 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_v2
The duckdb destination used duckdb:////Users/alainamartinez/Desktop/de-zoomcamp/de-zoomcamp/workshops/dlt_workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1712307820.7893162 is LOADED and contains no failed jobs




In [83]:
conn.sql("SELECT * FROM people_v2").df()

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1712307792.6355572,GmyOAhlvTl+vkw,None
1,2,Person_2,27,City_A,1712307792.6355572,WvU+btVlC36sow,None
2,7,Person_7,37,City_B,1712307820.7893162,/xB7NCxIfHqBaA,Job_7
3,3,Person_3,33,City_B,1712307820.7893162,pVpF+yHna7ZQxw,Job_3
4,6,Person_6,36,City_B,1712307820.7893162,2Y2qIfs0J2cP+Q,Job_6
5,8,Person_8,38,City_B,1712307820.7893162,9Su9Durfw6Wljw,Job_8
6,4,Person_4,34,City_B,1712307820.7893162,5gk35E+BS9iEmw,Job_4
7,5,Person_5,35,City_B,1712307820.7893162,y3erjK25CASE8w,Job_5


In [84]:
conn.sql("SELECT sum(age) FROM people_v2").df()

,sum(age)
0,266.0


### old didnt work lol - oh wait it did, there's supposed to be 8 rows. lol

In [69]:
# Importing the DLT library
import dlt

# Create a DLT pipeline for the first generator `people_1`
# The pipeline is set to load data into a DuckDB database with the dataset named 'people'
ppl_1_pipeline_final = dlt.pipeline(destination='duckdb', dataset_name='ppl_v2')

# Run the pipeline for the first generator, creating or replacing the table 'people'
info = ppl_1_pipeline_final.run(people_1(),
                             table_name="ppl_v2",
                             write_disposition="replace",
                            primary_key="id")



print(f"{info}\n\n")

Pipeline dlt_ipykernel_launcher load step completed in 0.16 seconds
1 load package(s) were loaded to destination duckdb and into dataset ppl_v2
The duckdb destination used duckdb:////Users/alainamartinez/Desktop/de-zoomcamp/de-zoomcamp/workshops/dlt_workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1712238697.3175511 is LOADED and contains no failed jobs




In [70]:
conn = duckdb.connect(f"{ppl_1_pipeline_final.pipeline_name}.duckdb")
display(conn.sql("show tables"))


┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

In [54]:
display(conn.sql("SELECT * FROM people_final").df())

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1712238452.307327,jO+1NllxfpDYxw
1,2,Person_2,27,City_A,1712238452.307327,g5rra64ZRHCqHw
2,3,Person_3,28,City_A,1712238452.307327,20ihQqzah6NRKQ
3,4,Person_4,29,City_A,1712238452.307327,EPBH2fyu7UChpQ
4,5,Person_5,30,City_A,1712238452.307327,jdkRUPCQT+4lig


merge

In [56]:
merge_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_final')

# run the pipeline with default settings, and capture the outcome
info = merge_pipeline.run(people_2(),
										table_name="people_final",
										write_disposition="merge",
                    primary_key='id')


In [57]:
display(conn.sql("SELECT * FROM people_final").df())

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1712238452.307327,jO+1NllxfpDYxw,None
1,2,Person_2,27,City_A,1712238452.307327,g5rra64ZRHCqHw,None
2,8,Person_8,38,City_B,1712238557.139104,YluunYAgIHQC2w,Job_8
3,3,Person_3,33,City_B,1712238557.139104,RZP4tO6kRhSKDw,Job_3
4,6,Person_6,36,City_B,1712238557.139104,iRpfinicp3/XJw,Job_6
5,5,Person_5,35,City_B,1712238557.139104,mqT4HEAbbHomRA,Job_5
6,7,Person_7,37,City_B,1712238557.139104,YQLNM8W1H7wQ8Q,Job_7
7,4,Person_4,34,City_B,1712238557.139104,F4Zz+93JnTHhPA,Job_4


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX